### Two different methods: 
- One csv file, which still has to be split into training and test data
- Two csv files, which are already split into training and test data

In [ ]:
# Define attribute columns here
case_attr = "Case ID"
act_attr = "concept:name"
time_attr = "Complete Timestamp"
path = "data/BPI_Challenge_2012_end.csv"

#### One csv file

In [ ]:
#path = "Data/sub_dataset.csv"
#baseline_log = LogFile(path, ",", 0, None, time_attr='event time:timestamp', trace_attr=case_attr,
#                    activity_attr=act_attr, convert=False, k=3)

#train_base_log, test_base_log = baseline_log.splitTrainTest(65, split_case=False, method="test-train")

In [ ]:
#path = "Data/BPI_Challenge_2012-test.csv"

#baseline_log = LogFile(path, ",", 0, None, time_attr='event time:timestamp', trace_attr=case_attr,
#                    activity_attr=act_attr, convert=False, k=3)

#train_base_log, test_base_log = baseline_log.splitTrainTest(70, split_case=False, method="test-train")

#### Two csv files

In [ ]:
#path_train = "Data/sub_data_train.csv" 
#path_test = "Data/sub_data_test.csv"

#path_train = 'Data/BPI_Challenge_2012-training.csv'
#path_test = 'Data\BPI_Challenge_2012-test.csv'

train_base_log = LogFile(path, ",", 0, None, time_attr=time_attr, trace_attr=case_attr,
                   activity_attr=act_attr, convert=False, k=3)
test_base_log = LogFile(path, ",", 0, None, time_attr=time_attr, trace_attr=case_attr,
                    activity_attr=act_attr, convert=False, k=3)

## Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
# only keep the starting and end point of a case 
def split_train_test(path, split_interval):
    df = pd.read_csv(path)
    data = df[(df['concept:name'] == 'A_SUBMITTED-COMPLETE') | (df['concept:name'] == 'End-End')]
    data['time:timestamp'] = pd.to_datetime(data['time:timestamp'])


    
    loss = len(data)
    
    for i in split_interval:
        train, test = train_test_split(data['Case ID'].unique(), test_size=(100-i)/100, shuffle=False)
        train_data = data[data['Case ID'].isin(train)]
        test_data = data[data['Case ID'].isin(test)]
        
        overlap = train_data[train_data['Complete Timestamp'] > test_data['Complete Timestamp'].min()][['Case ID']]
        
        if len(overlap) < loss:
            loss = len(overlap)
            best_train = train_data[~train_data['Case ID'].isin(overlap['Case ID'].tolist())]
            best_test = test_data
    
    
        print('Train data lost due to overlap: ' + str(len(overlap)/len(train_data)))
        return best_train, best_test

In [ ]:
train, test = split_train_test('data/BPI_2012_Converted.csv', range(67, 73))